In [27]:
from langchain.document_loaders import PyPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [28]:
loader = PyPDFLoader("./isma_merged_2.pdf")

In [29]:
data = loader.load()

In [37]:
print (f'Hay {len(data)} documentos')
print (f'Hay {len(data[0].page_content)} caracteres en el documento')

Hay 11 documentos
Hay 4213 caracteres en el documento


In [47]:
for i in range(len(data)):
    print(f"Pagina {i + 1}: " + str(data[i].page_content.replace('\n', ' ')))

Pagina 1: Experiencia Formación C F G S  D e s a r r o l l o  d e  A p l i c a c i o n e s  M u l t i p l a t a f o r m a  ( D A M ) B a c h i l l e r a t o  C i e n t í f i c o  T e c n o l ó g i c oJunio 2023 Septiembre 2021A c t u a l m e n t e  c u r s a n d o  e n  E s c o l a  P i a I E S  E l s  T r e s  T u r o n sDaniel García Serrano Idiomas InteresesE s p a ñ o l   T e c n o l o g í a T e n d e n c i a s D e p o r t eConocimientos P H P ´ /  L a r a v e l P y t h o n C #  /  . N E T  C o r e J a v a S c r i p t ,  H T M L y  C S S S Q LPERSONAL Nombre D a n i e l  G a r c í a  S e r r a n o Dirección C \  C a m í  C a n  Q u i n t a n a 0 8 3 5 0 ,  A r e n y s  d e  M a r Número de teléfono + 3 4  6 4 0  2 4  6 9  2 0 Correo electrónico d a n i i e e l g s @ g m a i l . c o m Fecha de nacimiento 0 9 / 0 9 / 2 0 0 3 Género H o m b r e Nacionalidad E s p a ñ o l Permiso de conducir B Github h t t p s : / / g i t h u b . c o m / d a n i i e e l g s ? t a b = r e p o s i t o r 

In [39]:
#divide los textos en chunks segun el chunk_size
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [40]:
print (f'Tienes {len(texts)} documentos')

Tienes 18 documentos


In [41]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

In [8]:
PINECONE_API_KEY = 'fbb7ad14-46bd-4ebf-af06-6dfc5efb8c15'
PINECONE_API_ENV = 'asia-southeast1-gcp'

In [ ]:
#import openai

#openai.api_type = "azure"
#openai.api_key = "68c1cf8e9d5d4a368f57bed9b9a6cc4b"
#openai.api_base = "https://openaipasionaus.openai.azure.com/"
#openai.api_version = "2022-12-01"

#response = openai.Embedding.create(
#    input="Quien tiene mas experiencia?",
#    engine="textEmbeddingAda002"
#)
#embeddings = response['data'][0]['embedding']

In [48]:
#embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

#Embedding = lenguaje natural que convierte el lenguaje humano a vectores matemático

#https://python.langchain.com/en/latest/reference/modules/embeddings.html
import os
os.environ["OPENAI_API_TYPE"]  = 'azure'
os.environ["OPENAI_API_BASE"] = 'https://openaipasionaus.openai.azure.com/'
os.environ["OPENAI_API_KEY"] = '9cae1236e81e43138b8895ead77acb12'

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)

In [49]:
#Pinecone = base de datos de Vectores

pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "langchain2"

In [50]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

#From_texts = Embendding a los documentos y añade los documentos al indice proveido (langchain2)

In [51]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [52]:
import os
import openai
from langchain.llms import AzureOpenAI
from llama_index import LangchainEmbedding
from llama_index import GPTSimpleVectorIndex, SimpleDirectoryReader, LLMPredictor, PromptHelper

#os.environ["OPENAI_API_TYPE"]  = 'azure'
#os.environ["OPENAI_API_VERSION"]  = '2023-03-15-preview'
#os.environ["OPENAI_API_BASE"] = 'https://openaipasionaus.openai.azure.com/'
#os.environ["OPENAI_API_KEY"] = '9cae1236e81e43138b8895ead77acb12'

openai.api_type = "azure"
openai.api_version = "2023-03-15-preview" 
openai.api_base = 'https://openaipasionaus.openai.azure.com/' # Your Azure OpenAI resource's endpoint value.
openai.api_key = '9cae1236e81e43138b8895ead77acb12'

llm = AzureOpenAI(deployment_name="textDavinci03Model")
llm_predictor = LLMPredictor(llm=llm)

#llm = OpenAI(temperature=0, model="textDavinci03Model")
chain = load_qa_chain(llm, chain_type="stuff")

#Apuntes

# LLM es un tipo de modelo de inteligencia artificial (IA) que se utiliza para realizar tareas relacionadas con el procesamiento del lenguaje natural. 
# Los LLM suelen estar entrenados con grandes cantidades de texto y pueden ser utilizados para comprender y responder a consultas escritas en un idioma determinado.
#chain_type = stufff , carga una cadena que no pertenece a una categoria o tema especifico

USING API_BASE: 
https://openaipasionaus.openai.azure.com/


In [78]:
query = "Dime 2 personas con experiencia en c#"
docs = docsearch.similarity_search(query, include_metadata=True)

#busca en el documento por similaridad de la query

In [79]:
chain.run(input_documents=docs, question=query)

' David Salcedo Salamanca y Microsoft Azure Fundamentals.'